In [170]:
import pandas as pd 
import datetime as dt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import poisson
import math
import random

In [281]:
#df = pd.read_csv('../data/trip_cleaned_final.csv') 
df['pickup_time_act'] = pd.to_datetime(df['pickup_datetime']).dt.time
df['dropoff_time_act'] = pd.to_datetime(df['dropoff_datetime']).dt.time
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

df.shape

(13873809, 39)

In [171]:
p_df_org = pd.read_csv("../data/taxi_after_dropoff.csv")
p_df_org = p_df_org.rename(columns={"Unnamed: 0": "zone"})

In [172]:
#Check if all prob for a zone equal to 1
p_df_org.sum(axis = 1) - (p_df_org.index+1) 

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
258    1.0
259    1.0
260    1.0
261    1.0
262    1.0
Length: 263, dtype: float64

In [173]:
#p_df_org
p_df = p_df_org.drop(columns=["zone"])
p_df

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
0,0.001487,0.000000,0.000000,0.000929,0.0,0.0,0.000248,0.000000,0.0,0.000062,...,0.000000,0.000248,0.000124,0.000062,0.0,0.000000,0.000124,0.010407,0.000434,0.001487
1,0.000000,0.571429,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.071429,0.000000
2,0.000000,0.000000,0.020638,0.001876,0.0,0.0,0.001876,0.000000,0.0,0.000000,...,0.000000,0.001876,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.022514,0.058161
3,0.000000,0.000000,0.000000,0.299248,0.0,0.0,0.000209,0.000000,0.0,0.000015,...,0.000000,0.000539,0.000718,0.000000,0.0,0.000000,0.000105,0.001062,0.000853,0.001346
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.032258,0.000000,0.000000,0.0,0.000000,0.000000,0.032258,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.000000,0.000000,0.000000,0.004122,0.0,0.0,0.001649,0.000000,0.0,0.000000,...,0.001649,0.000824,0.000824,0.000000,0.0,0.021434,0.000000,0.001649,0.018137,0.073372
259,0.000069,0.000000,0.000000,0.001922,0.0,0.0,0.035562,0.000069,0.0,0.000343,...,0.000000,0.016751,0.003089,0.000000,0.0,0.000000,0.102156,0.000618,0.006934,0.008856
260,0.000015,0.000000,0.000000,0.001988,0.0,0.0,0.000104,0.000000,0.0,0.000059,...,0.000000,0.000252,0.000208,0.000000,0.0,0.000000,0.000045,0.318062,0.000282,0.000430
261,0.000000,0.000000,0.000000,0.001605,0.0,0.0,0.000507,0.000000,0.0,0.000026,...,0.000000,0.000204,0.000086,0.000000,0.0,0.000000,0.000132,0.000125,0.371068,0.145969


In [174]:

print(p_df.max(axis=1))
print(p_df.idxmax(axis=1))
p_list = list(p_df.iloc[0,:])
max(p_list)
print(p_list.index(max(p_list)) +1  )
p_df.columns


0      0.361767
1      0.571429
2      0.138837
3      0.319128
4      0.161290
         ...   
258    0.088211
259    0.219621
260    0.318062
261    0.371068
262    0.316449
Length: 263, dtype: float64
0      231
1        2
2      138
3       79
4      231
      ... 
258     74
259    138
260    261
261    262
262    263
Length: 263, dtype: object
231


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '254', '255', '256', '257', '258', '259', '260', '261', '262', '263'],
      dtype='object', length=263)

In [353]:
# This policy always reposition to highest prob 
def policy_best_zone(zone_in) :
    p_list = list(p_df.iloc[(zone_in-1),:])
    zone_repos = p_list.index(max(p_list))+1
    return(zone_repos)

# This policy always reposition to any prob > 3%
def policy_top_zone(zone_in) :
    zone_s = p_df.iloc[(zone_in-1),:]
    #Pick a destination zone with prob > 5% 
    #print(zone_s[zone_s>0.03].index)
    #print(random.choice(zone_s[zone_s>0.03].index))
    return (random.choice(zone_s[zone_s>0.03].index))

In [351]:
repos_zone = policy_best_zone(140)
print(repos_zone)

repos_zone = policy_top_zone(140)
print(repos_zone)

140
140


In [212]:
trip_df_org = pd.read_csv("../data/dropoff_after_pickup.csv")
#trip_df_org.head()
trip_df = trip_df_org.drop(columns=["Unnamed: 0"])
trip_df.head()

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
0,0.728571,0.000000,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
1,0.000000,0.727273,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.095238,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.095238,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,0.000640,0.000000,0.000020,0.02046,0.0,0.0,0.0042,0.00004,0.00008,0.0,...,0.000060,0.01182,0.01276,0.00062,0.00004,0.0001,0.00064,0.00444,0.00594,0.01074
4,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [266]:
# Code to calculate next zone 

#trip_df.head()
pick_zone = 79
zone_s = trip_df.iloc[(pick_zone-1),:]
#Pick a destination zone with prob > 5% 
print(zone_s[zone_s>0.03].index)
print(random.choice(zone_s[zone_s>0.03].index))
#high_rides = zone_1 > 0.05
#zone_1[high_rides]

Index(['79', '107', '148', '170', '186', '234', '249'], dtype='object')
170


In [179]:
# Check sum of prob = 1.0 
trip_df.sum(axis = 1)

0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
258    1.0
259    1.0
260    1.0
261    1.0
262    1.0
Length: 263, dtype: float64

In [180]:
tt_df = pd.read_csv("../data/trip_time_scr_dst_t.csv")
tt_df.shape


(471868, 7)

In [181]:
tt_df.tail()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
471863,47,23:30:00,263.0,257.0,35.833333,34.500000,6
471864,47,23:30:00,263.0,260.0,17.536667,16.983333,15
471865,47,23:30:00,263.0,261.0,25.850000,25.000000,7
471866,47,23:30:00,263.0,262.0,4.028523,3.316667,149
471867,47,23:30:00,263.0,263.0,4.614421,3.000000,193


In [319]:
dist_df = pd.read_csv("../data/trip_distance_scr_dst.csv")

In [320]:
repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        

,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,1.0,1.0,2.732745,0.700,51
1,1.0,43.0,0.500000,0.500,1
2,1.0,50.0,3.030000,3.030,1
3,1.0,70.0,24.700000,24.700,1
4,1.0,79.0,15.570000,15.570,1
...,...,...,...,...,...
28220,263.0,259.0,11.204483,10.955,58
28221,263.0,260.0,5.177234,5.090,282
28222,263.0,261.0,7.713881,7.945,286
28223,263.0,262.0,0.688296,0.600,8057


In [182]:
fare_df = pd.read_csv("../data/fare_amount_scr_dst_t.csv")
fare_df.shape

(471868, 7)

In [183]:
fare_df.head()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,8.666667,6.0,3
1,0,00:00:00,3.0,69.0,24.500000,24.5,1
2,0,00:00:00,4.0,4.0,7.175000,4.5,40
3,0,00:00:00,4.0,7.0,21.305556,20.5,18
4,0,00:00:00,4.0,13.0,14.437500,14.0,16


In [184]:
start_time = 0
start_zone = random.randint(1, 263)

print(start_time, start_zone)

0 216


In [209]:
trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == 34) & (tt_df["dropoff_taxizone_id"] ==35) ), "mean"] #.describe().mean()
print(trip_duration)

Series([], Name: mean, dtype: float64)


In [345]:
# Simulation for shift of 8 hours 
start_time = 0

prob = 0.6
random.seed(1)
cur_ran_zone = random.randint(1, 263) # decide later if we want to fix it for all episodes

earnings_list = []

for i in range(0,10) :
    cur_zone = cur_ran_zone
    print("episode", i, cur_zone)
    cur_time = start_time
    trip_completion = 0
    earnings = 0
    while(cur_time < start_time+17):
    
        #check cur taxi zone and cur_time and find matching prob 
        match_prob = prob 

        if (match_prob >= 0.9 ) :
            pickup_time = random.randint(1, 4)
        elif (match_prob >= 0.7  ):
            pickup_time = random.randint(5, 10)
        elif (match_prob >= 0.5  ):
            pickup_time = random.randint(11, 15)
        elif (match_prob >= 0.3  ):
            pickup_time = random.randint(16, 20)
        elif (match_prob >= 0.1  ):
            pickup_time = random.randint(21, 29)
    
   
    
        # Choose a ride, choose a destination zone 
        zone_s = trip_df.iloc[cur_zone-1,:]
        top_dest_list = list(zone_s[zone_s>0.03].index)
        #print(top_dest_list)
        dest_zone = int(random.choice(top_dest_list))
        #trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone) & (tt_df["pickup_datetime_index"]==cur_time)), "mean"].values
        #print(type(cur_zone), type(dest_zone))
        trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        #trip_duration = 0
        #print(trip_duration)
        trip_fare = fare_df.loc[((fare_df["pickup_taxizone_id"] == cur_zone) & (fare_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        fuel_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == cur_zone) & (dist_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean())*0.124

        earnings = earnings + trip_fare - fuel_cost
        
        trip_completion = trip_completion + pickup_time + trip_duration
        print(cur_time, pickup_time, cur_zone, dest_zone, trip_duration, trip_completion, trip_fare, fuel_cost,earnings)
    
        if (trip_completion >= 30 ):
            trip_completion = trip_completion - 30 
            cur_time = cur_time + 1
    
        # call policy now to reposition 
        repos_zone = policy_best_zone(dest_zone)
        repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        
        print("repos_cost:, repos_zone ", repos_cost, repos_zone)
    
        cur_zone = repos_zone
        earnings = earnings-repos_cost
        
    earnings_list.append(earnings)
    
    

episode 0 69
0 15 69 42 10.842646840837695 25.842646840837695 10.0866876774395 0.23811220779220785 9.848575469647292
repos_cost:, repos_zone  0.12557967838444284 41
0 13 41 42 10.840123451967951 49.68277029280564 11.191601832466404 0.15256078425800923 20.762036839471243
repos_cost:, repos_zone  0.12557967838444284 41
1 14 41 236 13.119549389674596 46.80231968248024 13.003280782310487 0.2411813803339519 33.39855656306334
repos_cost:, repos_zone  0.0940940447657661 236
2 14 236 263 9.316512755870553 40.11883243835079 9.855230926373114 0.09618858288893661 43.063504861781745
repos_cost:, repos_zone  0.10047076316437956 263
3 14 263 141 9.939752574343734 34.05858501269453 10.42324338625448 0.12293512658180132 53.26334235829004
repos_cost:, repos_zone  0.1031067202684312 141
4 11 141 237 10.112903552406367 25.171488565100894 10.252221723754904 0.10030071558278283 63.312156646193735
repos_cost:, repos_zone  0.09645421424365419 237
4 11 237 236 9.880906232814112 46.05239479791501 10.5331184533

0 11 69 244 10.540280691770983 21.54028069177098 10.053414875531633 0.31772987012987014 9.735685005401763
repos_cost:, repos_zone  0.14494745843230408 244
0 15 244 238 16.65342049891035 53.19370119068133 17.781923524233427 0.47143595982142844 26.901225111381457
repos_cost:, repos_zone  0.10191158972958211 238
1 14 238 142 10.916720633374236 48.110421824055564 11.43296706176108 0.16854564608797143 38.06373493732499
repos_cost:, repos_zone  0.10244989010988981 142
2 12 142 161 14.102981229729952 44.21340305378551 12.727850500673842 0.1763822759339016 50.512753271955035
repos_cost:, repos_zone  0.09761027672160101 161
3 15 161 237 11.166864809577055 40.38026786336257 11.118695030705322 0.13758123874185912 61.39625678719689
repos_cost:, repos_zone  0.09645421424365419 237
4 12 237 141 9.931259787613564 32.31152765097613 10.18578016765943 0.10122825352450154 71.38435448708817
repos_cost:, repos_zone  0.1031067202684312 141
5 15 141 162 12.454776938219759 29.76630458919589 11.733340322029491

13 13 230 48 11.136889345236565 48.272099296872625 10.690615594402384 0.10450013043603262 194.20691021903878
repos_cost:, repos_zone  0.1058040890295201 48
14 12 48 161 13.12798927101002 43.40008856788265 11.79130323021202 0.12885702091055115 205.7635523393107
repos_cost:, repos_zone  0.09761027672160101 161
15 13 161 237 11.166864809577055 37.566953377459704 11.118695030705322 0.13758123874185912 216.64705585455255
repos_cost:, repos_zone  0.09645421424365419 237
16 12 237 163 11.98764614237077 31.554599519830475 11.358748497925856 0.12639246033800505 227.78295767789675
repos_cost:, repos_zone  0.10863306141275703 163
episode 6 69
0 14 69 69 10.271604897614823 24.27160489761482 9.478522284717148 0.1633847619047619 9.315137522812385
repos_cost:, repos_zone  0.25101142857142855 74
0 13 74 236 12.215148711730219 49.486753609345044 12.35053027060396 0.21429494749356065 21.200361417351356
repos_cost:, repos_zone  0.0940940447657661 236
1 11 236 237 10.71510368525675 41.201857294601794 10.9

6 14 236 237 10.71510368525675 36.9732244373497 10.932601779596112 0.13308258442306226 98.78598440603885
repos_cost:, repos_zone  0.09645421424365419 237
7 13 237 236 9.880906232814112 29.854130670163812 10.533118453377067 0.13253667220676 109.09011197296552
repos_cost:, repos_zone  0.0940940447657661 236
7 11 236 141 11.221753801368504 52.07588447153232 11.164733657469352 0.14001945284879708 120.02073213282031
repos_cost:, repos_zone  0.1031067202684312 141
8 13 141 263 9.361202384172476 44.43708685570479 10.231210725787848 0.13190696373962427 130.0169291746001
repos_cost:, repos_zone  0.10047076316437956 263
9 14 263 75 9.831885633985431 38.26897248969022 10.429200243419144 0.12735820082856567 140.21830045402632
repos_cost:, repos_zone  0.11778289485436402 75
10 13 75 236 9.630625650577581 30.899598140267802 10.366176441602214 0.1197921416796192 150.34690185909454
repos_cost:, repos_zone  0.0940940447657661 236
11 15 236 239 11.768142328527254 27.667740468795056 11.78678556498773 0.1

In [346]:
print(earnings_list)
earnings_avg = sum(earnings_list) / len(earnings_list)
print(earnings_avg)

[226.3963355999, 236.96839452533993, 230.89413092378848, 245.0435121292332, 235.7230071814249, 227.674324616484, 251.480070497484, 230.17616934833373, 243.5439723353877, 264.59800993808096]
239.2497927095457


In [347]:
# Simulation for shift of 8 hours 
start_time = 0

prob = 0.6
random.seed(1)
cur_ran_zone = random.randint(1, 263) # decide later if we want to fix it for all episodes

earnings_list = []

for i in range(0,10) :
    cur_zone = int(cur_ran_zone)
    print("episode", i, cur_zone)
    cur_time = start_time
    trip_completion = 0
    earnings = 0
    while(cur_time < start_time+17):
    
        #check cur taxi zone and cur_time and find matching prob 
        match_prob = prob 

        if (match_prob >= 0.9 ) :
            pickup_time = random.randint(1, 4)
        elif (match_prob >= 0.7  ):
            pickup_time = random.randint(5, 10)
        elif (match_prob >= 0.5  ):
            pickup_time = random.randint(11, 15)
        elif (match_prob >= 0.3  ):
            pickup_time = random.randint(16, 20)
        elif (match_prob >= 0.1  ):
            pickup_time = random.randint(21, 29)
    
   
    
        # Choose a ride, choose a destination zone 
        zone_s = trip_df.iloc[cur_zone-1,:]
        top_dest_list = list(zone_s[zone_s>0.03].index)
        #print(top_dest_list)
        dest_zone = int(random.choice(top_dest_list))
        #trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone) & (tt_df["pickup_datetime_index"]==cur_time)), "mean"].values
        #print(type(cur_zone), type(dest_zone))
        trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        #trip_duration = 0
        #print(trip_duration)
        trip_fare = fare_df.loc[((fare_df["pickup_taxizone_id"] == cur_zone) & (fare_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        fuel_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == cur_zone) & (dist_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean())*0.124

        earnings = earnings + trip_fare - fuel_cost
 
        
        trip_completion = trip_completion + pickup_time + trip_duration
        print(cur_time, pickup_time, cur_zone, dest_zone, trip_duration, trip_completion, trip_fare, earnings)
    
        if (trip_completion >= 30 ):
            trip_completion = trip_completion - 30 
            cur_time = cur_time + 1
    
        # call policy now to reposition 
        repos_zone = policy_top_zone(dest_zone)
        repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        
        print("repos_cost:, repos_zone ", repos_cost, repos_zone)
    
    
        cur_zone = int(repos_zone)
        earnings = earnings-repos_cost
        
    earnings_list.append(earnings)

episode 0 69
0 15 69 42 10.842646840837695 25.842646840837695 10.0866876774395 9.848575469647292
repos_cost:, repos_zone  0.23655395784543312 75
0 11 75 263 9.721046346961346 46.56369318779904 10.376647050088518 19.863385050480723
repos_cost:, repos_zone  0.10047076316437956 263
1 14 263 236 9.379111468369024 39.942804656168065 10.031034326101205 29.700919840518914
repos_cost:, repos_zone  0.09618858288893661 263
2 14 263 141 9.939752574343734 33.8825572305118 10.42324338625448 39.90503951730266
repos_cost:, repos_zone  0.1031067202684312 141
3 14 141 140 9.981285194281813 27.863842424793614 10.205523385817951 49.90786126477227
repos_cost:, repos_zone  0.13288442026082134 263
3 14 263 229 13.044608022936451 54.90845044773007 12.752032366549363 62.31687902112516
repos_cost:, repos_zone  0.10159756557355545 233
4 11 233 230 14.78634940473608 50.69479985246615 12.71064487684641 74.78141789749868
repos_cost:, repos_zone  0.10602737017726768 161
5 13 161 164 11.375676961263759 45.0704768137

0 11 74 75 9.822122711244567 45.66476955208226 10.395296278605521 19.965148253713977
repos_cost:, repos_zone  0.11778289485436402 75
1 11 75 141 12.265391934845729 38.93016148692799 12.337141420494056 31.979369214304374
repos_cost:, repos_zone  0.13190696373962427 263
2 15 263 263 9.183337615632967 33.11349910256096 10.014068436149678 41.76105992355005
repos_cost:, repos_zone  0.09302877289863591 236
3 15 236 161 15.632316981841637 33.74581608440259 13.999457904250658 55.4362231432346
repos_cost:, repos_zone  0.09926806757648018 230
4 15 230 170 13.7105655472527 32.45638163165529 12.256811115991876 67.43902214915784
repos_cost:, repos_zone  0.1183121452154106 162
5 15 162 237 10.886513026839785 28.342894658495076 10.943440671669673 78.1327599244374
repos_cost:, repos_zone  0.1499934303297466 43
5 14 43 239 10.889076363938702 53.23197102243378 11.284689122209986 89.10458329522498
repos_cost:, repos_zone  0.11137482580275986 143
6 14 143 48 11.815452876081554 49.04742389851533 11.5883534

repos_cost:, repos_zone  0.10450013043603262 48
episode 6 69
0 13 69 69 10.271604897614823 23.27160489761482 9.478522284717148 9.315137522812385
repos_cost:, repos_zone  0.4626262857142857 263
0 15 263 236 9.379111468369024 47.65071636598385 10.031034326101205 18.79051679030067
repos_cost:, repos_zone  0.09618858288893661 263
1 13 263 161 16.989012727385898 47.639729093369745 15.022567613766164 33.443971150980325
repos_cost:, repos_zone  0.09773479976750384 162
2 15 162 162 10.592720604227955 43.2324496975977 10.755366865733443 43.999944536150416
repos_cost:, repos_zone  0.10165868079584854 162
3 12 162 162 10.592720604227955 35.82517030182566 10.755366865733443 54.55199404029216
repos_cost:, repos_zone  0.10165868079584854 162
4 11 162 164 12.557040760136486 29.382211061962145 11.633297484638945 65.94263419407395
repos_cost:, repos_zone  0.10536877832389997 100
4 14 100 161 12.068401666379412 55.450612728341554 11.316067406120954 77.02844119832305
repos_cost:, repos_zone  0.0992680675

3 12 236 262 10.403060921878021 26.672712654422895 10.4624399905179 53.900152879183
repos_cost:, repos_zone  0.17810018679063597 237
3 12 237 141 9.931259787613564 48.60397244203646 10.18578016765943 63.80660460652729
repos_cost:, repos_zone  0.13650442949998573 262
4 15 262 237 12.685156420457673 46.28912886249413 12.082006887150559 75.57400687738723
repos_cost:, repos_zone  0.1499934303297466 43
5 13 43 163 12.408480600677429 41.69760946317156 11.81550816800981 87.07966471107098
repos_cost:, repos_zone  0.10513258100756892 161
6 14 161 236 13.252042859938419 38.94965232310998 12.94556166399352 99.68875649627218
repos_cost:, repos_zone  0.1509974111478179 43
7 12 43 238 10.97087294533212 31.9205252684421 11.645734169457928 111.01083478732038
repos_cost:, repos_zone  0.10276296433505892 151
8 15 151 142 12.328201116846047 29.248726385288144 12.672778613515007 123.35626641852934
repos_cost:, repos_zone  0.15863284238765923 43
8 15 43 162 14.010054238922953 58.2587806242111 12.8535425763

In [348]:
print(earnings_list)
earnings_avg_2 = sum(earnings_list) / len(earnings_list)
print(earnings_avg_2)

[240.21148326996087, 230.75365487236152, 239.40222049783515, 233.73241652281567, 240.87956737315736, 231.1067286387574, 233.41821500483124, 234.17358224766272, 239.91464737763445, 235.68296214126514]
235.92754779462817
